In [40]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

In [8]:
url = "https://yahoo-finance15.p.rapidapi.com/api/yahoo/qu/quote/AAPL/cashflow-statement"

In [11]:
headers = {
    'x-rapidapi-host': "yahoo-finance15.p.rapidapi.com",
    'x-rapidapi-key': "aaf8b66fefmsh0021683065f6bddp18ea51jsn7b4e9b5a44cd"
    }

In [22]:
response = requests.request("GET", url, headers=headers)

In [24]:
jsonResponse = response.json()

In [25]:
jsonResponse

{'cashflowStatementHistory': {'cashflowStatements': [{'maxAge': 1,
    'endDate': {'raw': 1569628800, 'fmt': '2019-09-28'},
    'netIncome': {'raw': 55256000000,
     'fmt': '55.26B',
     'longFmt': '55,256,000,000'},
    'depreciation': {'raw': 12547000000,
     'fmt': '12.55B',
     'longFmt': '12,547,000,000'},
    'changeToNetincome': {'raw': 5076000000,
     'fmt': '5.08B',
     'longFmt': '5,076,000,000'},
    'changeToAccountReceivables': {'raw': 245000000,
     'fmt': '245M',
     'longFmt': '245,000,000'},
    'changeToLiabilities': {'raw': -2548000000,
     'fmt': '-2.55B',
     'longFmt': '-2,548,000,000'},
    'changeToInventory': {'raw': -289000000,
     'fmt': '-289M',
     'longFmt': '-289,000,000'},
    'changeToOperatingActivities': {'raw': -896000000,
     'fmt': '-896M',
     'longFmt': '-896,000,000'},
    'totalCashFromOperatingActivities': {'raw': 69391000000,
     'fmt': '69.39B',
     'longFmt': '69,391,000,000'},
    'capitalExpenditures': {'raw': -10495000000

In [30]:
data=response.json()

In [46]:
df = json_normalize(data["cashflowStatements"])

KeyError: 'cashflowStatements'

In [48]:
pd.DataFrame(data)

,cashflowStatementHistory,cashflowStatementHistoryQuarterly
cashflowStatements,"[{'maxAge': 1, 'endDate': {'raw': 1569628800, ...","[{'maxAge': 1, 'endDate': {'raw': 1593216000, ..."
maxAge,86400,86400
